In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_circles
from sklearn.metrics import accuracy_score, log_loss
from tqdm import tqdm

In [127]:
def initialisation(n0,n1,n2, nCouche):

    WMultiple = []
    bMultiple = []
    WMultiple.append(np.random.randn(n1, n0))
    bMultiple.append(np.zeros((n1, 1)))
    for i in range(0, nCouche-2):
        WMultiple.append(np.random.randn(n1, n1))
        bMultiple.append(np.zeros((n1, 1)))
    WMultiple.append(np.random.randn(n2, n1))
    bMultiple.append(np.zeros((n2, 1)))

    return WMultiple, bMultiple

In [76]:
def forward_propagation(X, parametres):

    #on récupère les paramètres pour nos 2 couches
    WMultiple, bMultiple = parametres
    AMultiple = []
    for i in range(len(WMultiple)):
            #on calcule les activations de la première couche 
        if i == 0:
            ZMultiple = WMultiple[i].dot(X) + bMultiple[i]
            AMultiple.append(1 / (1 + np.exp(-ZMultiple)))
        else:
             #on calcule les activations de la deuxième couche à partir de l'activation de la première couche
            ZMultiple = WMultiple[i].dot(AMultiple[-1]) + bMultiple[i]
            AMultiple.append(1 / (1 + np.exp(-ZMultiple)))
        
    return AMultiple

In [131]:
def back_propagation(X, y, parametres, activations):

    WMultiple = parametres[0]

    # A1 = activations['A1']
    # A2 = activations['A2']
    Wlast = WMultiple[-1]

    m = y.shape[1]

    dZMultiple = []
    dbMultiple = []
    dWMultiple = []
    
    
    
    for i in range(len(WMultiple)-1, -1, -1):
        print("i",i)
        aIndex = i-1
        if i == (len(WMultiple)-1):
            dZMultiple.append(activations[i] - y)
            dWMultiple.append(1 / m * dZMultiple[-1].dot(activations[-2].T))
            dbMultiple.append(1 / m * np.sum(dZMultiple[-1], axis=1, keepdims = True))
        elif i == 0:
            print("W shape",WMultiple[1].shape)
            print("dZ shape",dZMultiple[-1].shape)
            #pour la premiere couche on a besoin des poids de la deuxieme couche, du coup l'index 1 de WMultiple
            dZMultiple.append(np.dot(WMultiple[1].T, dZMultiple[-1]) * activations[i] * (1 - activations[i]))
            dWMultiple.append(1 / m * dZMultiple[-1].dot(X.T))
            dbMultiple.append(1 / m * np.sum(dZMultiple[-1], axis=1, keepdims = True))
        else:
            dZMultiple.append(np.dot(WMultiple[-i].T, dZMultiple[-1]) * activations[aIndex] * (1 - activations[aIndex]))
            dWMultiple.append(1 / m * dZMultiple[-1].dot(activations[aIndex].T))
            dbMultiple.append(1 / m * np.sum(dZMultiple[-1], axis=1, keepdims = True))


    # Avant de retourner les gradients, on inverse l'ordre des listes
    dWMultiple = dWMultiple[::-1]
    dbMultiple = dbMultiple[::-1]
    #affichage de chaque gradient
    for i in range(len(WMultiple)):
        print("WMultiple",WMultiple[i].shape)
        
    for i in range(len(dWMultiple)):
        print("dWMultiple",dWMultiple[i].shape) 
    
    for i in range(len(dbMultiple)):
        print("dbMultiple",dbMultiple[i].shape)
    # dZ3 = A3 - y
    # dW3 = 1 / m * dZ3.dot(A2.T)
    # db3 = 1 / m * np.sum(dZ3, axis=1, keepdims = True)
    
    # dZ2 = np.dot(Wt.T, dZ3) * A2 * (1 - A2)

    # dZ1 = np.dot(W2.T, dZ2) * A1 * (1 - A1)
    # dW1 = 1 / m * dZ1.dot(X.T)
    # db1 = 1 / m * np.sum(dZ1, axis=1, keepdims = True)

    gradients = {
        'dW' : dWMultiple,
        'db' : dbMultiple,
        
    }
    
    return gradients

In [115]:
def update(gradients, parametres, learning_rate):

    WMultiple = parametres[0]
    bMultiple = parametres[1]

    dW = gradients['dW']
    db = gradients['db']

    for i in range(len(WMultiple)):
        WMultiple[i] = WMultiple[i] - learning_rate * dW[i]
        bMultiple[i] = bMultiple[i] - learning_rate * db[i]


    parametres = {
        'W': WMultiple,
        'b': bMultiple,
    }

    return parametres

In [5]:
def predict(X, parametres):
  activations = forward_propagation(X, parametres)
  A2 = activations[-1]
  return A2 >= 0.5

In [52]:
def  neural_network(X, y, n1=32, nCouche = 3,learning_rate = 0.1, n_iter = 100):


     # initialisation parametres
    n0 = X.shape[0]
    n2 = y.shape[0]
    np.random.seed(0)
    parametres = initialisation(n0,n1,n2, nCouche)

    train_loss = []
    train_acc = []
    history = []

    # gradient descent
    for i in tqdm(range(n_iter)):
        activations = forward_propagation(X, parametres)
        lastActivation = activations[-1]

        # Plot courbe d'apprentissage
        train_loss.append(log_loss(y.flatten(), lastActivation.flatten()))
        y_pred = predict(X, parametres)
        train_acc.append(accuracy_score(y.flatten(), y_pred.flatten()))
        
        history.append([tuple(parametres), train_loss, train_acc, i])

        # mise a jour
        gradients = back_propagation(X, y, parametres, activations)
        parametres = update(gradients, parametres, learning_rate)


    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='train loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label='train acc')
    plt.legend()
    plt.show()

    return parametres

# Dataset
fictif

In [ ]:
X, y = make_circles(n_samples=1000, noise=0.1, factor=0.3, random_state=0)
X = X.T
y = y.reshape((1, y.shape[0]))

print('dimensions de X:', X.shape)
print('dimensions de y:', y.shape)

plt.scatter(X[0, :], X[1, :], c=y, cmap='summer')
plt.show()

In [132]:
print("NN - X shape", X.shape)
print("NN - y shape", y.shape)
neural_network(X, y, n1=32, n_iter=10000)

NN - X shape (2, 1000)
NN - y shape (1, 1000)


  0%|          | 1/10000 [00:00<01:10, 141.65it/s]

i 2
i 1
i 0
W shape (32, 32)
dZ shape (32, 1000)
WMultiple (32, 2)
WMultiple (32, 32)
WMultiple (1, 32)
dWMultiple (32, 2)
dWMultiple (32, 32)
dWMultiple (1, 32)
dbMultiple (32, 1)
dbMultiple (32, 1)
dbMultiple (1, 1)


AttributeError: 'str' object has no attribute 'dot'